<a href="https://colab.research.google.com/github/hungngo97/time-series-stock-market-analysis/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
TODO: 
Step 1: Build dataset
  - Probably 10 companies should be enough but try to gather from different fields
  - Each company gather its 3 year(?) stock end of date history + related info 
  such as Description, Financial data such as high/low/Pricebook
  ==> Each company is a row but stock data is a dataframe?
  - Collect another data table for S&P 500 to keep track of local economy 
  to predict company's performance (https://datahub.io/core/s-and-p-500#readme)
  ==> Try to match similar time from S&P500 for company stock data above
  
  


"""

In [0]:
!pip install quandl

     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 61kB 19.1MB/s 
     |████████████████████████████████| 2.3MB 14.5MB/s 
     |████████████████████████████████| 102kB 25.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: more-itertools 7.0.0
    Uninstalling more-itertools-7.0.0:
      Successfully uninstalled more-itertools-7.0.0


# Section 1: Reading Local Data from Google Drive

In [0]:
# Code to read csv file into Colaboratory:

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.9MB/s 


In [0]:
filenames = ['appl.csv', 'dow.csv', 'ge.csv', 'googl.csv', 
            'msft.csv', 'ndaq.csv', 'qcom.csv', 'sbux.csv',
            'sp500.csv']
links = ['https://drive.google.com/open?id=16nTgI_dCcp_sE9D_9p4rGpnduor7TinZ',
         'https://drive.google.com/open?id=1uGoY6WdxCikPQUQMkWT200MiL_osZl8p',
        'https://drive.google.com/open?id=1gObLhrQAKjTuRRyUoQNM0i8WUyzPWaER',
        'https://drive.google.com/open?id=1d0xSZw0IBzsyKWe0QUO8ELkZNsuJZTaz',
        'https://drive.google.com/open?id=1YfIzYhvPi93lmWiu3kRp4_3ZXHrIlpKj',
        'https://drive.google.com/open?id=1898MYq3yIl4HifdCLmjHbvxI-rZAMJyx',
        'https://drive.google.com/open?id=1Uc1i8-1-y_rrUnRWf_NMknm6hI5XUROr',
        'https://drive.google.com/open?id=1saxPuHUE_FxUzvLfAmCag6__Xdp0APw4',
        'https://drive.google.com/open?id=1USDDk1ydtnOya_5mpBEus9u0AymEYr_6']
for index, filename in enumerate(filenames):
  _, id = links[index].split('=')
  print (id) # Verify that you have everything after '='
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(filename)

16nTgI_dCcp_sE9D_9p4rGpnduor7TinZ
1uGoY6WdxCikPQUQMkWT200MiL_osZl8p
1gObLhrQAKjTuRRyUoQNM0i8WUyzPWaER
1d0xSZw0IBzsyKWe0QUO8ELkZNsuJZTaz
1YfIzYhvPi93lmWiu3kRp4_3ZXHrIlpKj
1898MYq3yIl4HifdCLmjHbvxI-rZAMJyx
1Uc1i8-1-y_rrUnRWf_NMknm6hI5XUROr
1saxPuHUE_FxUzvLfAmCag6__Xdp0APw4
1USDDk1ydtnOya_5mpBEus9u0AymEYr_6


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

appl = pd.read_csv('appl.csv')
dow = pd.read_csv('dow.csv')
ge = pd.read_csv('ge.csv')
googl = pd.read_csv('googl.csv')
msft = pd.read_csv('msft.csv')
ndaq = pd.read_csv('ndaq.csv')
qcom = pd.read_csv('qcom.csv')
sbux = pd.read_csv('sbux.csv')
sp500 = pd.read_csv('sp500.csv')

In [0]:
# import needed libraries
import quandl
import pandas as pd

# add quandl API key for unrestricted
quandl.ApiConfig.api_key = 'uJ_ygibX-DzsaJnx5fCd'

# get the table for daily stock prices and,
# filter the table for selected tickers, columns within a time range
# set paginate to True because Quandl limits tables API to 10,000 rows per call
data = quandl.get_table('WIKI/PRICES', ticker = ['SPX', 'MSFT', 'WMT'], 
                        qopts = { }, 
                        date = { 'gte': '2018-01-01', 'lte': '2018-05-31' }, 
                        paginate=True)

# create a new dataframe with 'date' column as index
new = data.set_index('date')

# use pandas pivot function to sort adj_close by tickers
clean_data = new.pivot(columns='ticker')

In [0]:
clean_data.head

<bound method NDFrame.head of               open             high                low            close  \
ticker        MSFT     WMT     MSFT       WMT     MSFT       WMT   MSFT   
date                                                                      
2018-01-02  86.125   99.30  86.3100   99.7900  85.5000   98.5200  85.95   
2018-01-03  86.055   98.85  86.5100  100.0000  85.9700   98.7700  86.35   
2018-01-04  86.590   99.54  87.6600   99.7800  86.5700   98.9110  87.11   
2018-01-05  87.660   99.64  88.4100  100.3800  87.4300   99.6100  88.19   
2018-01-08  88.200  100.25  88.5800  101.7500  87.6046  100.2100  88.28   
2018-01-09  88.650  102.03  88.7272  102.3500  87.8600  100.3100  88.22   
2018-01-10  87.860   99.75  88.1900   99.8900  87.4100   98.5300  87.82   
2018-01-11  88.130   99.70  88.1300  100.4500  87.2400   98.7800  88.08   
2018-01-12  88.670  100.39  89.7800  101.4400  88.4500  100.3000  89.58   
2018-01-16  90.100  101.34  90.7900  101.9100  88.0104  100.3400  88.3